This script is aiming to find the proper script to make a classicification of the images.

# Import pacakage

In [1]:

import tensorflow as tf

import numpy as np

import warnings # current version of seaborn generates a bunch of warnings that we'll ignore
warnings.filterwarnings("ignore")
# import seaborn as sns
#import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import imshow
import numpy as np
import PIL
#import requests
from io import BytesIO
from IPython.display import display
%matplotlib inline

#from mpl_toolkits.mplot3d import Axes3D
# from sklearn import datasets
# from sklearn.decomposition import PCA
# from sklearn.utils import shuffle

# tf.enable_eager_execution()

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Activation

import os
import scipy

# Load Images

In [2]:
# Get the name of all the labels
home_path = '/home/bowen/Documents/Ecomed/'
data_path = '/home/bowen/Documents/Ecomed/dataset'
class_names = os.listdir(data_path)
class_names

['background',
 'needles_I_V_Cannula',
 'needles_safety',
 'needles_black',
 'needles_yellow',
 'garbage_gloves',
 'syringes_large',
 'garbage_alcohol_pad',
 'needles_butterfly',
 'syringes_small',
 'needles_pink',
 'syringes_1ml',
 'garbage_cotton_ball',
 'needles_blood_collection']

In [3]:
# Set up data generators that can read images from our dataset into Keras.
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        validation_split=0.2)

train_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=12,
        class_mode='binary',
        subset='training')

validation_generator = datagen.flow_from_directory(
        data_path,
        target_size=(224, 224),
        batch_size=12,
        class_mode='binary',
        subset='validation')

Found 1631 images belonging to 14 classes.
Found 402 images belonging to 14 classes.


In [4]:
# # Show several images
# # data, labels = validation_generator.next()

# # for i, sample in enumerate(data):
#     plt.imshow(sample)
#     print(class_names[int(labels[i])])
#     plt.show()

# MobileNet Model

In [23]:
mobilenet = tf.keras.applications.MobileNetV2()
mobilenet.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 225, 225, 3)  0           input_7[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu

# Train the Model

In [24]:
opt = tf.keras.optimizers.SGD(0.001)
#opt = tf.train.GradientDescentOptimizer(0.001)
mobilenet.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
checkpoint_path = os.path.join(home_path + 'models', "model2.h5")
cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_path)

In [25]:
mobilenet.fit_generator(generator=train_generator, 
                    validation_data=validation_generator, 
                    epochs=100,
                    callbacks=[cp_callback])

Epoch 1/100
136/136 [==============================] - 93s 684ms/step - loss: 4.7729 - acc: 0.2282 - val_loss: 8.0520 - val_acc: 0.0000e+00
Epoch 2/100
136/136 [==============================] - 24s 178ms/step - loss: 1.6661 - acc: 0.6610 - val_loss: 6.3552 - val_acc: 0.0050
Epoch 3/100
136/136 [==============================] - 24s 180ms/step - loss: 0.8829 - acc: 0.7909 - val_loss: 5.2595 - val_acc: 0.0199
Epoch 4/100
136/136 [==============================] - 25s 182ms/step - loss: 0.6233 - acc: 0.8320 - val_loss: 4.3091 - val_acc: 0.1592
Epoch 5/100
136/136 [==============================] - 24s 178ms/step - loss: 0.4706 - acc: 0.8713 - val_loss: 3.3810 - val_acc: 0.2512
Epoch 6/100
136/136 [==============================] - 25s 181ms/step - loss: 0.3811 - acc: 0.9006 - val_loss: 2.6219 - val_acc: 0.3706
Epoch 7/100
136/136 [==============================] - 24s 178ms/step - loss: 0.3420 - acc: 0.9147 - val_loss: 2.3992 - val_acc: 0.4279
Epoch 8/100
136/136 [=======================

Epoch 61/100
136/136 [==============================] - 24s 178ms/step - loss: 0.0164 - acc: 0.9963 - val_loss: 0.0982 - val_acc: 0.9577
Epoch 62/100
136/136 [==============================] - 24s 177ms/step - loss: 0.0247 - acc: 0.9945 - val_loss: 0.0886 - val_acc: 0.9751
Epoch 63/100
136/136 [==============================] - 24s 177ms/step - loss: 0.0251 - acc: 0.9957 - val_loss: 0.0999 - val_acc: 0.9677
Epoch 64/100
136/136 [==============================] - 24s 180ms/step - loss: 0.0202 - acc: 0.9957 - val_loss: 0.0810 - val_acc: 0.9801
Epoch 65/100
136/136 [==============================] - 24s 178ms/step - loss: 0.0241 - acc: 0.9951 - val_loss: 0.1209 - val_acc: 0.9652
Epoch 66/100
136/136 [==============================] - 24s 178ms/step - loss: 0.0242 - acc: 0.9938 - val_loss: 0.1248 - val_acc: 0.9577
Epoch 67/100
136/136 [==============================] - 24s 175ms/step - loss: 0.0241 - acc: 0.9939 - val_loss: 0.1455 - val_acc: 0.9627
Epoch 68/100
136/136 [===================

In [27]:
tf.contrib.saved_model.save_keras_model(mobilenet, home_path + 'models')


Consider using a TensorFlow optimizer from `tf.train`.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['serving_default']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/bowen/Documents/Ecomed/models/temp-b'1550686223'/saved_model.pb


b'/home/bowen/Documents/Ecomed/models/1550686223'

In [28]:
tf.keras.models.save_model(mobilenet, 
                           os.path.join(home_path + 'models', 'model_no_compile.h5'),
                           include_optimizer=False)